# Metaclasses

Metaclasses provide customization of class instantiation.

## Creating a class the 'hard' way

In [56]:
def init(self, name):
    self.name = name

def say(self):
    print(self.name)
    
attrs = {
    '__init__': init,
    'say': say,
    'kingdom': 'catdom'
}

superclasses = tuple([object])
Cat = type('Cat', superclasses, attrs)

In [58]:
a = Cat('name')
a.say()

name


## Workflow of metaclasses

In [6]:
import pprint as pp
import textwrap

In [53]:
class MyMeta(type):
    
    def __call__(cls, *args, **kwargs):
        print('MyMeta: __call__ start')
        indent = '    '
        print(textwrap.indent('arg0: {}'.format(cls), indent))
        print(textwrap.indent('args: {}'.format(pp.pformat(args)), indent))
        print(textwrap.indent('kwargs: {}'.format(pp.pformat(kwargs)), indent))
        to_return = super(MyMeta, cls).__call__(*args, **kwargs)
        print('MyMeta: __call__ end')
        return to_return

    def __new__(cls, name, superclasses, dct):
        '''
        Creates an instance of the class MyMeta
        cls is an instance of the type (meta)class--i.e. MyMeta
        
        The way we call type in the 'hard' way above, the arguments for this __new__
        method mirror that.
        
        name is the name of the class
        parents is a tuple of superclasses
        dct denote the attributes of the class
        
        Theory: This is basically a way for us to 'decorate' the type function.
        
        '''
        
        print('MyMeta: __new__ start')
        indent = '    '
        print(textwrap.indent('arg0: {}'.format(cls), indent))
        print(textwrap.indent('arg1: {}'.format(name), indent))
        print(textwrap.indent('arg2: {}'.format(superclasses), indent))
        print(textwrap.indent('arg2: {}'.format(pp.pformat(dct)), indent))
            
        to_return = super(MyMeta, cls).__new__(cls, name, superclasses, dct)
        
        # At this point, to_return is now a fully fledged class!
        # As such, you could start adding attributes to the class like you would
        # in the __init__ method.
        to_return.attr_from_new = 'attr_from_new'
        
        print(textwrap.indent('returning: {}'.format(to_return), indent))
        
        print('MyMeta: __new__ end')
        return to_return  # common mistake is to not return anything and just call the superclass

    
    def __init__(cls, name, superclasses, dct):
        '''
        
        By this time, the class has been created.
        We can initialize a class the same way we initialize an instance!
        
        cls is the return value of __new__
        
        '''
        print('MyMeta: __init__ start')
        indent = '    '        
        print(textwrap.indent('arg0: {}'.format(cls), indent))
        print(textwrap.indent('arg1: {}'.format(name), indent))
        print(textwrap.indent('arg2: {}'.format(superclasses), indent))
        print(textwrap.indent('arg2: {}'.format(pp.pformat(dct)), indent))

        cls.attr_from_init = 'attr_from_init'
        
        # Questionable if this following line is necessary?
        super(MyMeta, cls).__init__(name, superclasses, dct)
        
        print('MyMeta: __init__ end')

In [54]:
class Foo(metaclass=MyMeta):
    
    FOO_CLASS_VARIABLE = 'BAR'
        
    def __new__(cls, *args, **kwargs):
        '''
        Creates an instance of the class Foo
        cls is an instance of the MyMeta (meta) class--i.e. Foo
        '''
        
        print('Foo: __new__ start')
        obj = super(Foo, cls).__new__(cls, *args, **kwargs)
        obj.attr = 42
        indent = '    '
        print(textwrap.indent('arg0: {}'.format(cls), indent))
        print(textwrap.indent('args: {}'.format(pp.pformat(args)), indent))
        print(textwrap.indent('kwargs: {}'.format(pp.pformat(kwargs)), indent))
        print(textwrap.indent('returning: {}'.format(obj), indent))
        print('Foo: __new__ end')
        return obj
    
    def __init__(self):
        '''
        Initializes the instance.
        self is an instance of the class Foo
        '''
        print('Foo: __init__ start')
        print(textwrap.indent('arg0: {}'.format(str(self)), '    '))
        print('Foo: __init__ end')
        

MyMeta: __new__ start
    arg0: <class '__main__.MyMeta'>
    arg1: Foo
    arg2: ()
    arg2: {'FOO_CLASS_VARIABLE': 'BAR',
     '__classcell__': <cell at 0x103abcd08: empty>,
     '__init__': <function Foo.__init__ at 0x103ba2048>,
     '__module__': '__main__',
     '__new__': <function Foo.__new__ at 0x103b701e0>,
     '__qualname__': 'Foo'}
    returning: <class '__main__.Foo'>
MyMeta: __new__ end
MyMeta: __init__ start
    arg0: <class '__main__.Foo'>
    arg1: Foo
    arg2: ()
    arg2: {'FOO_CLASS_VARIABLE': 'BAR',
     '__classcell__': <cell at 0x103abcd08: MyMeta object at 0x7fe331429018>,
     '__init__': <function Foo.__init__ at 0x103ba2048>,
     '__module__': '__main__',
     '__new__': <function Foo.__new__ at 0x103b701e0>,
     '__qualname__': 'Foo'}
MyMeta: __init__ end


In [55]:
a = Foo()

MyMeta: __call__ start
    arg0: <class '__main__.Foo'>
    args: ()
    kwargs: {}
Foo: __new__ start
    arg0: <class '__main__.Foo'>
    args: ()
    kwargs: {}
    returning: <__main__.Foo object at 0x103b65c18>
Foo: __new__ end
Foo: __init__ start
    arg0: <__main__.Foo object at 0x103b65c18>
Foo: __init__ end
MyMeta: __call__ end


In [132]:
Foo.attr_from_new

'attr_from_new'

In [131]:
Foo.attr_from_init

'attr_from_init'

In [35]:
Foo.FOO_CLASS_VARIABLE

'BAR'

In [40]:
a.attr_from_new

'attr_from_new'

In [41]:
a.attr_from_init

'attr_from_init'

In [42]:
a.attr

42